In [1]:
import os
import shutil
import logging
import tabulate
import cloudpickle as pickle
import pandas as pd
import numpy as np
from functools import partial
import time
from pathlib import Path
import datamol as dm
from IPython.display import clear_output
from tempfile import NamedTemporaryFile, TemporaryDirectory
from itertools import combinations
from joblib import Parallel, delayed
from rdkit import Chem
from rdkit.Chem import rdFMCS
import concurrent.futures
from spyrmsd import rmsd, molecule
from espsim import GetEspSim
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN

import oddt
import oddt.fingerprints
import oddt.shape
import oddt.toolkits.rdk

from HandsFreeDocking.Wrapper_Docking import PipelineDocking
from HandsFreeDocking.analysis.clustering import (
    OptimizedDBSCANClustering, PairwiseMatrixComputer, OptimizedKMedoidsClustering,
    calc_rmsd_mcs_with_timeout, calc_usr_similarity, calc_splif
)

from HandsFreeDocking.tools.Protein_Minimization import ProteinMinimizer
from HandsFreeDocking.analysis.clustering import OptimizedHierarchicalClustering

In [2]:
protein_pdb = Path("./examples/LAG3_Moloc_2.pdb")
ligands_sdf = Path("./examples/Ligands_To_Dock.sdf")
cystal_sdf = Path("./examples/Fake_Crystal.sdf")

docking_pkl = Path("./examples/TMP_Docking/docking_results.pkl")
docking_dir = Path("./examples/TMP_Docking")
docking_dir_RXDOCK = Path("./examples/TMP_Docking_RXDOCK")
docking_pkl_RXDOCK = docking_dir_RXDOCK / "docking_results.pkl"

docking_dir_PLANTS = Path("./examples/TMP_Docking_PLANTS")
docking_pkl_PLANTS = docking_dir_PLANTS / "docking_results.pkl"

# Check if all files exist
files_to_check = [protein_pdb, ligands_sdf, cystal_sdf]
all_files_exist = all(file.exists() for file in files_to_check)

RERUN = True

from HandsFreeDocking.RxDock_Pipeline import RxDock_Docking
from HandsFreeDocking.Plants_Pipeline import Plants_Docking

rxdock_pipeline = RxDock_Docking(docking_dir_RXDOCK, protein_pdb, cystal_sdf, ligands_sdf, toolkit="openeye")
rxdock_pipeline.main(n_poses=10, n_cpus=2)

plants_pipeline = Plants_Docking(docking_dir_PLANTS, protein_pdb, cystal_sdf, ligands_sdf, toolkit="openeye")
plants_pipeline.main(n_confs=10, n_cpus=2)

---

In [3]:
if RERUN == True:
    shutil.rmtree(docking_dir, ignore_errors=True)
    
    # Initialize the docking pipeline
    docking = PipelineDocking(
        workdir=docking_dir,
        docking_software=["plants", "rxdock"],      # Choose one or more: "plants", "gnina", "openeye"
        settings=(10, 4),                                     # (n_conformers, n_cpus)
        protein_pdb=protein_pdb,
        ligands_input=ligands_sdf,                            # Can be SDF or SMILES file
        crystal_sdf=cystal_sdf,
        toolkit="openeye"                                     # Choose "cdpkit" or "openeye"
    )

    # Run the docking and get results
    results = docking.run()
    FULL_DF = docking.concat_df()

    with open(docking_pkl, "wb") as f:
        pickle.dump(FULL_DF, f)

    clear_output()
else:
    with open(docking_pkl, "rb") as f:
        FULL_DF = pickle.load(f)

In [4]:
results["rxdock"]

,CHROM.1,Chiral_Atoms,Chiral_ID,Name,RI,Rbt.Current_Directory,Rbt.Executable,Rbt.Library,Rbt.Parameter_File,Rbt.Receptor,...,SCORE.SYSTEM.DIHEDRAL,SCORE.SYSTEM.norm,SCORE.heavy,SCORE.norm,LIGAND_ENTRY,Software,ID,Molecule,Protein_Path,ChiralInfo
0,"84.42531504,-53.87238081,55.13280041,-22.11318...",,Stereo_0,6-methylflavone_Iso0,80,/home/hitesit/Python_Packages/Docking_Pipeline...,rbdock/0.1.0,librxdock.so/0.1.0,/home/hitesit/Software/rxdock/rxdock_installat...,/home/hitesit/Python_Packages/Docking_Pipeline...,...,0,0,18,-0.892887,6-methylflavone_Iso0_RxDock-P1,rxdock,6-methylflavone_Iso0_RxDock-P1,<rdkit.Chem.rdchem.Mol object at 0x7fcbd8fe2570>,/home/hitesit/Python_Packages/Docking_Pipeline...,NaN
1,"-11.02213624,-51.84540555,52.12013660,-18.9910...",,Stereo_0,6-methylflavone_Iso0,80,/home/hitesit/Python_Packages/Docking_Pipeline...,rbdock/0.1.0,librxdock.so/0.1.0,/home/hitesit/Software/rxdock/rxdock_installat...,/home/hitesit/Python_Packages/Docking_Pipeline...,...,0,0,18,-0.833597,6-methylflavone_Iso0_RxDock-P2,rxdock,6-methylflavone_Iso0_RxDock-P2,<rdkit.Chem.rdchem.Mol object at 0x7fcbd8fe3d10>,/home/hitesit/Python_Packages/Docking_Pipeline...,NaN
2,"169.22256227,-51.87540079,52.12415391,-18.9783...",,Stereo_0,6-methylflavone_Iso0,80,/home/hitesit/Python_Packages/Docking_Pipeline...,rbdock/0.1.0,librxdock.so/0.1.0,/home/hitesit/Software/rxdock/rxdock_installat...,/home/hitesit/Python_Packages/Docking_Pipeline...,...,0,0,18,-0.83282,6-methylflavone_Iso0_RxDock-P3,rxdock,6-methylflavone_Iso0_RxDock-P3,<rdkit.Chem.rdchem.Mol object at 0x7fcaf3f23bc0>,/home/hitesit/Python_Packages/Docking_Pipeline...,NaN
3,"173.31374090,-51.88832126,52.19300716,-19.0620...",,Stereo_0,6-methylflavone_Iso0,80,/home/hitesit/Python_Packages/Docking_Pipeline...,rbdock/0.1.0,librxdock.so/0.1.0,/home/hitesit/Software/rxdock/rxdock_installat...,/home/hitesit/Python_Packages/Docking_Pipeline...,...,0,0,18,-0.831673,6-methylflavone_Iso0_RxDock-P4,rxdock,6-methylflavone_Iso0_RxDock-P4,<rdkit.Chem.rdchem.Mol object at 0x7fcaf3f214d0>,/home/hitesit/Python_Packages/Docking_Pipeline...,NaN
4,"-7.19226528,-51.89103024,52.16009046,-19.04080...",,Stereo_0,6-methylflavone_Iso0,80,/home/hitesit/Python_Packages/Docking_Pipeline...,rbdock/0.1.0,librxdock.so/0.1.0,/home/hitesit/Software/rxdock/rxdock_installat...,/home/hitesit/Python_Packages/Docking_Pipeline...,...,0,0,18,-0.831071,6-methylflavone_Iso0_RxDock-P5,rxdock,6-methylflavone_Iso0_RxDock-P5,<rdkit.Chem.rdchem.Mol object at 0x7fcaf3f21230>,/home/hitesit/Python_Packages/Docking_Pipeline...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,"-140.88892349,147.12060986,-1.02795550,21.9018...",,Stereo_0,Fisetin_Iso0,80,/home/hitesit/Python_Packages/Docking_Pipeline...,rbdock/0.1.0,librxdock.so/0.1.0,/home/hitesit/Software/rxdock/rxdock_installat...,/home/hitesit/Python_Packages/Docking_Pipeline...,...,0,0,21,-0.775322,Fisetin_Iso0_RxDock-P6,rxdock,Fisetin_Iso0_RxDock-P6,<rdkit.Chem.rdchem.Mol object at 0x7fcbd903eab0>,/home/hitesit/Python_Packages/Docking_Pipeline...,NaN
76,"116.19847791,41.77311688,117.06739974,-146.209...",,Stereo_0,Fisetin_Iso0,80,/home/hitesit/Python_Packages/Docking_Pipeline...,rbdock/0.1.0,librxdock.so/0.1.0,/home/hitesit/Software/rxdock/rxdock_installat...,/home/hitesit/Python_Packages/Docking_Pipeline...,...,0,0,21,-0.763845,Fisetin_Iso0_RxDock-P7,rxdock,Fisetin_Iso0_RxDock-P7,<rdkit.Chem.rdchem.Mol object at 0x7fcbd903eb20>,/home/hitesit/Python_Packages/Docking_Pipeline...,NaN
77,"-84.70579551,-31.30959856,-179.02372537,4.5037...",,Stereo_0,Fisetin_Iso0,80,/home/hitesit/Python_Packages/Docking_Pipeline...,rbdock/0.1.0,librxdock.so/0.1.0,/home/hitesit/Software/rxdock/rxdock_installat...,/home/hitesit/Python_Packages/Docking_Pipeline...,...,0,0,21,-0.687647,Fisetin_Iso0_RxDock-P8,rxdock,Fisetin_Iso0_RxDock-P8,<rdkit.Chem.rdchem.Mol object at 0x7fcbd903eb90>,/home/hitesit/Python_Packages/Docking_Pipeline...,NaN
78,"114.16205591,-5.87749037,-53

In [7]:
pwd

'/home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking'

In [ ]:
FULL_DF = FULL_DF[~FULL_DF['ID'].str.contains('L17')]
lig_name_series = FULL_DF["ID"].str.split("_").str[0]

FULL_DF.insert(1, "Lig_Name", lig_name_series)

In [ ]:
ALL_MOLS = FULL_DF["Molecule"].tolist()

In [ ]:
# Compute the distance matrix using your existing PairwiseMatrixComputer
computer = PairwiseMatrixComputer(ALL_MOLS, n_jobs=8, timeout=60)
rmsd_func = partial(calc_rmsd_mcs_with_timeout, timeout=60)
distance_matrix = computer.compute_matrix(rmsd_func)

In [ ]:
# Initialize the hierarchical clustering
clustering = OptimizedHierarchicalClustering(
    linkage_methods=('ward', 'complete', 'average'),
    use_dimensionality_reduction=True,
    verbose=True
)

# Perform hierarchical clustering with automatic parameter optimization
labels = clustering.fit(distance_matrix)

In [ ]:
# Extract clusters at different distance thresholds
tight_clusters = clustering.get_clusters_by_distance(0.4)  # More stringent similarity
loose_clusters = clustering.get_clusters_by_distance(1.5)  # More relaxed similarity

# Compare different clustering solutions
print(f"Optimal clustering has {len(np.unique(labels))} clusters")
print(f"Tight clustering has {len(np.unique(tight_clusters))} clusters")
print(f"Loose clustering has {len(np.unique(loose_clusters))} clusters")

# custom_labels = clustering.get_clusters_constrained(
#     distance_threshold=0.4,
#     min_clusters=3,
#     max_clusters=10
# )
# print(f"Custom clustering has {len(np.unique(custom_labels))} clusters")

In [ ]:
FULL_DF["Type_3"] = tight_clusters

In [ ]:
from pymol import cmd
cmd.reinitialize()
cmd.load(protein_pdb)

def tmp_save(mol: Chem.rdchem.Mol):
    with NamedTemporaryFile(suffix=".sdf", delete=False) as f:
        dm.to_sdf(mol, f.name)
        return f.name

for cluster, df in FULL_DF.groupby("Type_3"):
    IDs = []
    for ndx, sub_df in df.groupby("Lig_Name"):
        sub_df.sort_values(by="Score", ascending=False, inplace=True)
        TOP = sub_df.iloc[0]
        
        ID = TOP["ID"]
        IDs.append(ID)
        TMP_MOL = tmp_save(TOP["Molecule"])
    
        cmd.load(TMP_MOL, ID)
    
    cmd.group(f"Cluster_{cluster}", " ".join(IDs))

cmd.save("TMP.pse")

## Paiwise Calculation

```python
clustering_kmed = OptimizedKMedoidsClustering(
    k_range=(2, 20),
    use_dimensionality_reduction=True,
    verbose=True
)
labels_kmed = clustering_kmed.fit(rmsd_matrix)
results_kmed = clustering_kmed.get_results()
```

```python
clustering_dbscan = OptimizedDBSCANClustering(
    eps_range=(0.5, 5.0, 0.5),
    min_samples_range=(2, 15),
    max_noise_percent=15.0,
    max_clusters = 10,
    use_dimensionality_reduction=True,
    verbose=True
)
labels_dbscan = clustering_dbscan.fit(rmsd_matrix)
results_dbscan = clustering_dbscan.get_results()
```

In [ ]:
computer = PairwiseMatrixComputer(ALL_MOLS, n_jobs=8, timeout=60)
rmsd_funct = partial(calc_rmsd_mcs_with_timeout, timeout=60)
rmsd_matrix = computer.compute_matrix(rmsd_funct)

In [ ]:
clustering_dbscan = OptimizedDBSCANClustering(
    eps_range=(0.5, 5.0, 0.5),
    min_samples_range=(2, 15),
    max_noise_percent=15.0,
    max_clusters = 10,
    use_dimensionality_reduction=True,
    verbose=True
)
labels_dbscan = clustering_dbscan.fit(rmsd_matrix)
results_dbscan = clustering_dbscan.get_results()

In [ ]:
FULL_DF["Cluster_DBSCAN"] = labels_dbscan

In [ ]:
from pymol import cmd
cmd.reinitialize()
cmd.load(protein_pdb)

def tmp_save(mol: Chem.rdchem.Mol):
    with NamedTemporaryFile(suffix=".sdf", delete=False) as f:
        dm.to_sdf(mol, f.name)
        return f.name

for cluster, df in FULL_DF.groupby("Cluster_DBSCAN"):
    IDs = []
    for ndx, sub_df in df.groupby("Lig_Name"):
        sub_df.sort_values(by="Score", ascending=False, inplace=True)
        TOP = sub_df.iloc[0]
        
        ID = TOP["ID"]
        IDs.append(ID)
        TMP_MOL = tmp_save(TOP["Molecule"])
    
        cmd.load(TMP_MOL, ID)
    
    cmd.group(f"Cluster_{cluster}", " ".join(IDs))

cmd.save("TMP.pse")

## Protein Minimization

In [ ]:
from HandsFreeDocking.tools.Protein_Minimization import ProteinMinimizer

```python
protein_minimizer = ProteinMinimizer(FULL_DF, "Molecule", "PDB_Path")
protein_minimizer(protein_pdb, protein_pdb)
```